In [287]:
import os
import json
import pandas as pd
import traceback

In [288]:
from langchain.chat_models import ChatOpenAI

In [289]:
from dotenv import load_dotenv

load_dotenv() 

False

In [290]:
Keys=os.getenv("Key")

In [291]:
llm=ChatOpenAI(openai_api_key=Keys,model_name="gpt-3.5-turbo",temperature=0.5)

In [292]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021168DB38B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021168DB92B0>, temperature=0.5, openai_api_key='sk-proj-RxitJZaEkgNDlHTUPFXjT3BlbkFJuu67bFRx1ux0HOjwqaUW', openai_proxy='')

In [293]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [294]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [295]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [296]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [297]:
print(Keys)

sk-proj-RxitJZaEkgNDlHTUPFXjT3BlbkFJuu67bFRx1ux0HOjwqaUW


In [298]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [299]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [300]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [301]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [302]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [303]:
file_path=r"C:\Users\31ash\Desktop\mcq_generator\data.txt"

In [304]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [305]:
print(TEXT)

ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coherent and contextually grounded interactions.

Furthermore, ChatGPT demonstrates impressive language generat

In [306]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [307]:
NUMBER=5 
SUBJECT="TRIGONOMETRY"
TONE="hard"

In [308]:
#setting up Token usage tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coh

In [309]:
print(f"total tokens:{cb.total_tokens}")
print(f"Prompt tokens:{cb.prompt_tokens}")
print(f"Completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total tokens:2232
Prompt tokens:1418
Completion tokens:814
total cost:0.0037549999999999997


In [310]:
quiz=response.get("quiz")

In [311]:
type(quiz)

str

In [312]:
quiz=json.loads(quiz)

In [313]:
quiz

{'1': {'mcq': 'In a right triangle, if the tangent of an angle is equal to 0.75, what is the value of the cosine of that angle?',
  'options': {'a': '0.6', 'b': '0.8', 'c': '0.4', 'd': '0.3'},
  'correct': 'a'},
 '2': {'mcq': 'Given a triangle with sides of lengths 5, 12, and 13 units, what is the measure of the smallest angle in the triangle?',
  'options': {'a': '30 degrees',
   'b': '45 degrees',
   'c': '60 degrees',
   'd': '90 degrees'},
  'correct': 'd'},
 '3': {'mcq': 'If sin(x) = 0.8 and x is in the first quadrant, what is the value of cos(x)?',
  'options': {'a': '0.6', 'b': '0.3', 'c': '0.4', 'd': '0.2'},
  'correct': 'b'},
 '4': {'mcq': 'For a triangle with angles measuring 30 degrees, 60 degrees, and 90 degrees, what is the ratio of the side opposite the 60-degree angle to the hypotenuse?',
  'options': {'a': '1:√3', 'b': '1:2', 'c': '1:√2', 'd': '√3:2'},
  'correct': 'a'},
 '5': {'mcq': 'If tan(x) = 1.5 and x is in the second quadrant, what is the value of sin(x)?',
  'op

In [314]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [315]:
quiz_table_data


[{'MCQ': 'In a right triangle, if the tangent of an angle is equal to 0.75, what is the value of the cosine of that angle?',
  'Choices': 'a: 0.6 | b: 0.8 | c: 0.4 | d: 0.3',
  'Correct': 'a'},
 {'MCQ': 'Given a triangle with sides of lengths 5, 12, and 13 units, what is the measure of the smallest angle in the triangle?',
  'Choices': 'a: 30 degrees | b: 45 degrees | c: 60 degrees | d: 90 degrees',
  'Correct': 'd'},
 {'MCQ': 'If sin(x) = 0.8 and x is in the first quadrant, what is the value of cos(x)?',
  'Choices': 'a: 0.6 | b: 0.3 | c: 0.4 | d: 0.2',
  'Correct': 'b'},
 {'MCQ': 'For a triangle with angles measuring 30 degrees, 60 degrees, and 90 degrees, what is the ratio of the side opposite the 60-degree angle to the hypotenuse?',
  'Choices': 'a: 1:√3 | b: 1:2 | c: 1:√2 | d: √3:2',
  'Correct': 'a'},
 {'MCQ': 'If tan(x) = 1.5 and x is in the second quadrant, what is the value of sin(x)?',
  'Choices': 'a: 0.4 | b: 0.6 | c: 0.8 | d: 0.2',
  'Correct': 'b'}]

In [316]:
quiz=pd.DataFrame(quiz_table_data)

In [317]:
quiz.to_csv("machinelearning.csv",index=False)

In [318]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_15_2024_23_20_07'